In [0]:
import jdk.jfr.Configuration;
import jdk.jfr.EventType;
import jdk.jfr.FlightRecorder;
import jdk.jfr.Recording;

In [0]:
var cs = Configuration.getConfigurations();
cs.stream().map(c -> c.getName()).forEach(c-> println("Configuration -> %s",c))

In [0]:
var conf = Configuration.getConfiguration("profile");
var settings = conf.getSettings();
settings.put("jdk.MethodTrace#enabled", "true");
settings.put("jdk.MethodTrace#stackTrace", "true");
settings.put("jdk.MethodTrace#threshold", "1 ms");
settings.put("jdk.MethodTrace#filter", "@org.jtaccuino.jshell.StopWatch");
settings.put("jdk.MethodTiming#enabled","true");
settings.put("jdk.MethodTiming#filter","@org.jtaccuino.jshell.StopWatch");
var rec = new Recording(settings);
rec.setToDisk(false);
display(rec.getSettings().entrySet())

In [0]:
@org.jtaccuino.jshell.StopWatch
double myMethod() throws Exception {
  var result = 0d;  
  for (int i=0; i<1000; i++) {
      result += Math.pow(i,2)*Math.sin(i)*Math.cos(i);
      result += Math.pow(i,4)*Math.cos(i)*Math.sin(i);
      result += Math.pow(i,6)*Math.sin(i)*Math.cos(i);
      Thread.sleep(10);
  }
  return result;
}

In [0]:
rec.start();

In [0]:
var result = myMethod()

In [0]:
rec.dump(Path.of(cwd + "/jfr.dump"));
rec.stop();

In [0]:
println("Result: %f", result)

In [0]:
import jdk.jfr.consumer.RecordedEvent;
import jdk.jfr.consumer.RecordingFile;

In [0]:
var events = RecordingFile.readAllEvents(Path.of(cwd + "/jfr.dump"));
events.size()

In [0]:
var histogramData = events.stream().collect(Collectors.groupingBy(event -> event.getEventType().getName(), Collectors.counting()));
histogramData

In [0]:
histogramData.get("jdk.MethodTiming")

In [0]:
histogramData.get("jdk.MethodTrace")

In [0]:
histogramData.get("jdk.CPULoad")

In [0]:
import javafx.scene.chart.BarChart;
import javafx.scene.chart.CategoryAxis;
import javafx.scene.chart.NumberAxis;
import javafx.scene.chart.XYChart;
import javafx.util.StringConverter;

In [0]:
var xAxis = new CategoryAxis();
xAxis.setLabel("Event Type");
var yAxis = new NumberAxis();
yAxis.setLabel("Count");

var barChart = new BarChart<>(xAxis, yAxis);
barChart.setTitle("JFR Event Type Distribution");
barChart.setLegendVisible(false);
barChart.setCategoryGap(5); 

In [0]:
var series = new XYChart.Series<String, Number>();
histogramData.entrySet().stream()
    .sorted(Map.Entry.<String, Long>comparingByValue().reversed())
    .forEach(entry -> series.getData().add(new XYChart.Data<>(entry.getKey(), entry.getValue())));
barChart.getData().add(series);

In [0]:
display(barChart)

In [0]:
var methodTiming = events.stream().filter(e -> e.getEventType().getName().equals("jdk.MethodTiming")).toList();
methodTiming.size();

In [0]:
methodTiming.forEach(e -> println(""+e));

In [0]:
var methodTrace = events.stream().filter(e -> e.getEventType().getName().equals("jdk.MethodTrace")).toList();
methodTrace.size();

In [0]:
methodTrace.forEach(e -> println(""+e));

In [0]:
var stacks = methodTrace.stream().map(e -> e.getStackTrace()).toList();
stacks.size();

In [0]:
display(stacks.get(0).getFrames().stream().map(f -> f.getMethod().getType().getName() + "." + f.getMethod().getName() +"-"+f.getLineNumber()).toList())

In [0]:
import javafx.scene.chart.LineChart;

In [0]:
var xAxis = new NumberAxis();
xAxis.setLabel("Time");
xAxis.setForceZeroInRange(false);

var yAxis = new NumberAxis();
yAxis.setLabel("CPU Load (%)");
yAxis.setLowerBound(0);
yAxis.setUpperBound(100);

var lineChart = new LineChart<Number,Number>(xAxis, yAxis);
lineChart.setTitle("Machine Total CPU Load Over Time");
lineChart.setCreateSymbols(false); // Good for dense time-series data

var formatter = DateTimeFormatter.ofPattern("HH:mm:ss");

xAxis.setTickLabelFormatter(new StringConverter<Number>() {
            @Override
            public String toString(Number object) {
                if (object == null) {
                    return "";
                }
                long millis = object.longValue();
                Instant instant = Instant.ofEpochMilli(millis);
                LocalDateTime dateTime = LocalDateTime.ofInstant(instant, ZoneId.systemDefault());
                return formatter.format(dateTime);
            }

            @Override
            public Number fromString(String string) {
                // Not needed for display purposes, but required by the interface
                return 0;
            }
        });

var series = new XYChart.Series<Number,Number>();
series.setName("Machine Total Load");

var machineTotal = events.stream()
    .filter(e -> e.getEventType().getName().equals("jdk.CPULoad"))
    .sorted(Comparator.comparing(RecordedEvent::getStartTime))
    .map(e -> new XYChart.Data<Number, Number>(e.getStartTime().toEpochMilli(), e.getDouble("machineTotal") * 100))
    .toList();

series.getData().addAll(machineTotal);
lineChart.getData().add(series);
display(lineChart);